In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv(r"C:\Users\devpr\Desktop\projects\Customer Trends\customer_shopping_behaviour.csv")
df.describe().round(2)

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.00,3900.00,3900.00,3863.00,3900.00
mean,1950.50,44.07,59.76,3.75,25.35
std,1125.98,15.21,23.69,0.72,14.45
min,1.00,18.00,20.00,2.50,1.00
25%,975.75,31.00,39.00,3.10,13.00
50%,1950.50,44.00,60.00,3.80,25.00
75%,2925.25,57.00,81.00,4.40,38.00
max,3900.00,70.00,100.00,5.00,50.00


##### *Creating a copy and Converting Column headers to snakecase for easy workaround*

In [13]:
df_clean = df.copy()
df_clean.columns  = df_clean.columns.str.lower().str.replace(' ','_').str.replace(r'[()/]','', regex = True)
df_clean.head()

,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


### Checking for Duplicates

In [19]:
df_clean.duplicated().sum()
# No Row level duplicates

np.int64(0)

In [21]:
df_clean['customer_id'].duplicated().sum()
#No Customer ID duplicates

np.int64(0)

### Checking for Null Values


In [23]:
df_clean.isna().sum()
#There are NULL values in review_rating. I am replacing these NULL values with category level median values

customer_id                0
age                        0
gender                     0
item_purchased             0
category                   0
purchase_amount_usd        0
location                   0
size                       0
color                      0
season                     0
review_rating             37
subscription_status        0
shipping_type              0
discount_applied           0
promo_code_used            0
previous_purchases         0
payment_method             0
frequency_of_purchases     0
dtype: int64

In [29]:
df_clean['review_rating'] = df_clean.groupby('category')['review_rating'].transform(lambda x : x.fillna(x.median()))
df_clean['review_rating'].isna().sum()
#Replaced null values with the category level medians

np.int64(0)

In [30]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount_usd     3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3900 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used         3900 non-null   

##### Map the frequency_of_purchases  with INTEGER datatype by converting text to numbers

In [37]:
df_clean['frequency_of_purchases'].unique()

array(['Fortnightly', 'Weekly', 'Annually', 'Quarterly', 'Bi-Weekly',
       'Monthly', 'Every 3 Months'], dtype=object)

In [38]:
frequency_map = {
    'Fortnightly' : 14,
    'Weekly' : 7,
    'Annually' : 365,
    'Quarterly' : 90,
    'Bi-Weekly' : 14,
    'Monthly' : 30,
    'Every 3 Months' : 90
}
df_clean['frequency_purchases_days'] = df_clean['frequency_of_purchases'].map(frequency_map)
df_clean['frequency_purchases_days'].unique()

array([ 14,   7, 365,  90,  30])

##### Creating age based category group and map customers for customer categorisation

In [54]:
age_groups = ['Young Adult','Adult', 'Middle aged','Senior']
df_clean['age_group'] = pd.qcut(df_clean['age'], q= 4, labels = age_groups)

df_clean.info()

## Cleaning and Transforming Data set is completed, lets save the data in CSV file (as backup) and then load the dataset into postgreSQL database table

In [69]:
df_clean.to_csv('customer_shopping_trends_clean.csv', index = False)
print(f' customer_shopping_trends_clean.csv saved successfully')

 customer_shopping_trends_clean.csv saved successfully


### Connect to postgreSQL database 'Customers'

In [70]:
!pip install psycopg2-binary sqlalchemy

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 2.9 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.7 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 5.2 MB/s  0:00:00


In [85]:
from sqlalchemy import create_engine, text
username = 'postgres'
password = 'password'
host = 'localhost'
port = '5432'
database = 'perfectdb'

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [86]:
table_name = 'orders'
df_clean.to_sql(
    name = table_name,
    con = engine,
    if_exists = 'replace',
    index = False

)
print("Table created and Data loaded successfully")

Table created and Data loaded successfully


In [89]:
query = 'SELECT * FROM orders'
pd.read_sql(query, engine)

,customer_id,age,gender,item_purchased,category,purchase_amount_usd,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,frequency_purchases_days,age_group
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,14,Middle aged
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,14,Young Adult
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,7,Middle aged
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,7,Young Adult
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,365,Middle aged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,No,No,32,Venmo,Weekly,7,Adult
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,No,No,41,Bank Transfer,Bi-Weekly,14,Middle aged
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,No,No,24,Venmo,Quarterly,90,Middle aged
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,No,No,24,Venmo,Weekly,7,Adult
